<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import fastai

In [ ]:
from fastai.text import * 

In [ ]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [ ]:
# For veracity
train = read_veracity('modifiedTrain_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [ ]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [ ]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [ ]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [ ]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.558799,4.282627,0.289588,02:08


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.047124,3.981765,0.326281,02:40


In [ ]:
learn.predict("This is a review about", n_words=10)

'This is a review about our people to stay safe & & wtf why these'

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (61190 items)
x: TextList
xxbos @hash_pragma @bbcworld xxmaj they wo n't ever have any success with me , on that issue . xxup :p,xxbos xxmaj unlike xxmaj indian authorities during 26 / 11 . @gauravcsawant,xxbos @mohan_velu xxmaj scriptures call for the murder of infidels . xxmaj it 's ignorant to xxup ignore those sections . xxmaj they exist . xxmaj do n't pretend it 's just the ppl,xxbos xxunk families,xxbos @jdmoss @reuters xxmaj we are their enemy . xxmaj eventually they all get together for their supremacy . xxmaj you just another conceded idiot .
y: CategoryList
False,False,False,False,True
Path: .;

Valid: LabelList (15298 items)
x: TextList
xxbos xxup xxunk @huffpostcanada @theheraldsun xxmaj you seem to be a bit of a weird fuck do n't you ?,xxbos @amalfarahuk @cemb_forum xxmaj expected from a country run by inbred princes that have the mental capacity of toddlers . a bunch of cry babies # xxup ksa,xxbos → http : / / t.co / xx

In [ ]:
data_clas.show_batch()

text,target
xxbos @smac5000 xxmaj xxunk ha ha ha a school boy socialist with a love for xxmaj hamas & & hatred for the xxmaj english ! \n xxmaj what time does social studies start ? \n @ejecter4579 i hate the scum that did that today and every other day but to generalise xxmaj islam as a terrorist culture is just plain xxunk \n @smac5000 xxmaj to learn who,True
"xxbos xxup @ap if he is an xxmaj iranian xxmaj refugee did he grow up in a xxmaj sunni or xxmaj shia family , is he a xxmaj xxunk , or a xxmaj kurd , or just one of xxmaj tony xxmaj abbott 's agents",False
xxbos xxup @ap really ? i do n't understand why xxmaj obama wo n't admit a terrorist can be a lone wolf . xxmaj but like xxmaj ft . xxmaj hood & & xxmaj moore xxmaj ok they wo n't say it ! xxup unreal,False
xxbos @guardian \n xxup islamic xxup scholars xxup should xxup come xxup forward xxup to xxup end ' xxup jihad ' ! \n xxup it xxup is xxup inhuman & & xxup would xxup destroy xxup the xxup credibility xxup of xxup islam !,False
xxbos xxmaj say xxmaj what ? @rt_com xxmaj exactly 7 months after the downing of a xxunk above xxmaj ukraine a hostage on xxunk news ? xxup are xxup you xxup kidding xxup me ? ! ? # xxunk # xxup mh17 # xxunk,False


In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.545347,0.515917,0.747287,03:00


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.482635,0.448644,0.789842,03:24


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.462013,0.430440,0.795398,05:44
1,0.436950,0.385132,0.823572,05:44
2,0.396391,0.346250,0.839391,05:45
3,0.332044,0.305540,0.860178,06:02
4,0.287620,0.291304,0.870245,06:00
5,0.269118,0.281969,0.875735,06:13
6,0.226753,0.278573,0.880311,05:55
7,0.192907,0.277659,0.880703,05:54
8,0.172337,0.277873,0.880703,06:01
9,0.201690,0.289966,0.881488,05:56


In [ ]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(test['label'], l)

0.6612616339193381

In [ ]:
f1_score(test['label'], l, average='macro')

0.5013555864234438

In [ ]:
learn.export('veracity.pkl')